In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import lognorm, weibull_min, gamma
import warnings
warnings.filterwarnings('ignore')

# Set plot aesthetics using Seaborn
sns.set_style("whitegrid")  # Replace plt.style.use
sns.set_context("paper", font_scale=1.5)

In [186]:
# Load 2019(pre-SEZ) and 2021 (post-SEZ) data
# Assuming your Excel files are named accordingly
pre_sez = pd.read_csv('data/sector51gurugram2021.csv', parse_dates=['Timestamp'])
post_sez = pd.read_csv('data/sector51gurugram2023.csv', parse_dates=['Timestamp'])

# Display basic information
print("Pre-SEZ Data (2021) Shape:", pre_sez.shape)
print("Post-SEZ Data (2023) Shape:", post_sez.shape)

Pre-SEZ Data (2021) Shape: (365, 19)
Post-SEZ Data (2023) Shape: (365, 19)


In [187]:
# Preview the data
print("\nPre-SEZ Data Preview:")
pre_sez.head()
pre_sez.columns


Pre-SEZ Data Preview:


Index(['Timestamp', 'PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO (µg/m³)',
       'NO2 (µg/m³)', 'NOx (ppb)', 'NH3 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)',
       'Ozone (µg/m³)', 'AT (°C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'RF (mm)',
       'TOT-RF (mm)', 'SR (W/mt2)', 'BP (mmHg)', 'VWS (m/s)'],
      dtype='object')

In [188]:
# Check for missing values
print("\nMissing Values in Pre-SEZ Data:")
print(pre_sez.isnull().sum())
print("\nMissing Values in Post-SEZ Data:")
print(post_sez.isnull().sum())


Missing Values in Pre-SEZ Data:
Timestamp          0
PM2.5 (µg/m³)      0
PM10 (µg/m³)       0
NO (µg/m³)         1
NO2 (µg/m³)        1
NOx (ppb)          0
NH3 (µg/m³)        1
SO2 (µg/m³)        0
CO (mg/m³)         0
Ozone (µg/m³)      0
AT (°C)            6
RH (%)             4
WS (m/s)           0
WD (deg)         115
RF (mm)            0
TOT-RF (mm)        0
SR (W/mt2)         1
BP (mmHg)        365
VWS (m/s)        365
dtype: int64

Missing Values in Post-SEZ Data:
Timestamp          0
PM2.5 (µg/m³)      0
PM10 (µg/m³)       0
NO (µg/m³)         1
NO2 (µg/m³)        1
NOx (ppb)          0
NH3 (µg/m³)        1
SO2 (µg/m³)        0
CO (mg/m³)         0
Ozone (µg/m³)      0
AT (°C)            1
RH (%)             6
WS (m/s)           0
WD (deg)           0
RF (mm)            0
TOT-RF (mm)        0
SR (W/mt2)         0
BP (mmHg)        365
VWS (m/s)        365
dtype: int64


In [189]:
def preprocess_data(df):
    """
    Preprocess air quality data by:
    1. Setting date as index
    2. Filling missing values using linear interpolation
    3. Removing any rows that still have missing values
    4. Ensuring pollutant values are numeric
    """
    # Copy to avoid modifying original
    df_clean = df.copy()
    
    # Set date as index if not already
    if 'Timestamp' in df_clean.columns:
        df_clean.set_index('Timestamp', inplace=True)
    
    # Select only pollution columns we need
    pollutants = ['PM2.5 (µg/m³)', 'NO2 (µg/m³)', 'SO2 (µg/m³)', 'PM10 (µg/m³)','NO (µg/m³)','CO (mg/m³)', 'Ozone (µg/m³)']
    df_clean = df_clean[pollutants]
    
    # Convert 'NA' strings to np.nan
    df_clean.replace('NA', np.nan, inplace=True)
    
    # Ensure all values are numeric
    for col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Fill missing values using linear interpolation
    df_clean = df_clean.interpolate(method='linear')
    
    # Drop any remaining rows with NaN values
    df_clean.dropna(inplace=True)
    
    return df_clean

# Apply preprocessing
pre_clean = preprocess_data(pre_sez)
post_clean = preprocess_data(post_sez)

# Check if preprocessing was successful
print("Pre-SEZ clean data shape:", pre_clean.shape)
print("Post-SEZ clean data shape:", post_clean.shape)


Pre-SEZ clean data shape: (365, 7)
Post-SEZ clean data shape: (365, 7)


In [190]:
pre_sez.columns

Index(['Timestamp', 'PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO (µg/m³)',
       'NO2 (µg/m³)', 'NOx (ppb)', 'NH3 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)',
       'Ozone (µg/m³)', 'AT (°C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'RF (mm)',
       'TOT-RF (mm)', 'SR (W/mt2)', 'BP (mmHg)', 'VWS (m/s)'],
      dtype='object')

In [191]:
print("\nPre-SEZ Summary Statistics:")
pre_clean.describe()


Pre-SEZ Summary Statistics:


,PM2.5 (µg/m³),NO2 (µg/m³),SO2 (µg/m³),PM10 (µg/m³),NO (µg/m³),CO (mg/m³),Ozone (µg/m³)
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,111.169425,11.421315,3.619178,218.481671,8.947671,0.924521,31.538548
std,73.285560,7.850084,3.497029,115.721985,7.674716,0.692774,13.904514
min,24.780000,1.090000,0.910000,35.160000,0.530000,0.250000,1.330000
25%,53.790000,6.290000,2.000000,115.600000,4.460000,0.440000,20.930000
50%,86.750000,9.980000,2.840000,205.490000,7.090000,0.640000,29.620000
75%,151.460000,14.600000,3.840000,298.090000,10.900000,1.220000,39.160000
max,542.860000,73.490000,39.880000,635.180000,80.280000,3.990000,95.960000


In [181]:
print("\nPost-SEZ Summary Statistics:")
post_clean.describe()


Post-SEZ Summary Statistics:


,PM2.5 (µg/m³),NO2 (µg/m³),SO2 (µg/m³),PM10 (µg/m³),NO (µg/m³),CO (mg/m³),Ozone (µg/m³)
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,115.695926,25.367551,2.997641,196.406607,19.412083,1.139300,33.759941
std,57.080458,18.775793,2.281548,96.485220,22.590411,1.025102,8.591613
min,13.379348,1.955694,0.378785,21.243478,1.139201,0.195625,3.403333
25%,76.003571,11.781806,2.034167,130.171875,6.334306,0.438021,29.187049
50%,108.760417,23.161778,2.463333,184.359420,10.514762,0.736319,35.067951
75%,142.897917,32.449062,3.114931,246.499653,22.250451,1.507986,35.819529
max,389.013768,179.464819,29.430417,616.342935,175.525797,6.178125,57.058833


In [182]:
def fit_distributions(data, pollutant):
    """
    Fit Lognormal, Weibull, and Gamma distributions to pollution data.
    Returns parameters and goodness-of-fit statistics.
    """
    # Get pollutant data
    values = data[pollutant].values
    
    # Initialize results dictionary
    results = {}
    
    # Fit Lognormal distribution
    shape_ln, loc_ln, scale_ln = stats.lognorm.fit(values)
    results['Lognormal'] = {
        'params': (shape_ln, loc_ln, scale_ln),
        'aic': stats.lognorm.nnlf((shape_ln, loc_ln, scale_ln), values) * 2 + 6  # AIC calculation
    }
    
    # Fit Weibull distribution
    shape_wb, loc_wb, scale_wb = stats.weibull_min.fit(values)
    results['Weibull'] = {
        'params': (shape_wb, loc_wb, scale_wb),
        'aic': stats.weibull_min.nnlf((shape_wb, loc_wb, scale_wb), values) * 2 + 6
    }
    
    # Fit Gamma distribution
    shape_g, loc_g, scale_g = stats.gamma.fit(values)
    results['Gamma'] = {
        'params': (shape_g, loc_g, scale_g),
        'aic': stats.gamma.nnlf((shape_g, loc_g, scale_g), values) * 2 + 6
    }
    
    return results

In [183]:
def chi_square_test(data, pollutant, dist_name, params):
    """
    Perform Chi-square goodness-of-fit test on the fitted distribution.
    Lower Chi-square values indicate better fit.
    """
    values = data[pollutant].values
    
    # Create histogram of observed values
    hist, bin_edges = np.histogram(values, bins=10, density=False)

#    hist, bin_edges = np.histogram(values, bins='auto', density=False)
    
    # Calculate bin midpoints for expected calculation
    bin_midpoints = (bin_edges[1:] + bin_edges[:-1]) / 2
    
    # Calculate expected frequencies based on the fitted distribution
    if dist_name == 'Lognormal':
        cdf_values = stats.lognorm.cdf(bin_edges, *params)
    elif dist_name == 'Weibull':
        cdf_values = stats.weibull_min.cdf(bin_edges, *params)
    elif dist_name == 'Gamma':
        cdf_values = stats.gamma.cdf(bin_edges, *params)
    
    # Calculate expected frequencies in each bin
    expected = len(values) * np.diff(cdf_values)
    
    # Handle zeros in expected frequencies (add a small value to avoid division by zero)
    expected = np.where(expected < 0.001, 0.001, expected)
    
    # Calculate Chi-square statistic
    chi2_stat = np.sum((hist - expected)**2 / expected)
    
    # Calculate degrees of freedom (bins - parameters - 1)
    df = len(hist) - len(params) - 1
    
    # Calculate p-value
    p_value = 1 - stats.chi2.cdf(chi2_stat, df)
    
    return {'chi2': chi2_stat, 'p_value': p_value, 'df': df}

In [184]:
# Define pollutants to analyze
pollutants = ['PM2.5 (µg/m³)','PM10 (µg/m³)','NO (µg/m³)','NO2 (µg/m³)','CO (mg/m³)','SO2 (µg/m³)','Ozone (µg/m³)']

# Initialize dictionaries to store fitting results
pre_distributions = {}
post_distributions = {}
chi_square_results = {}

# Fit distributions to each pollutant in pre-SEZ data
for pollutant in pollutants:
    pre_distributions[pollutant] = fit_distributions(pre_clean, pollutant)
    
    # Find best distribution based on AIC
    best_dist = min(pre_distributions[pollutant], 
                    key=lambda x: pre_distributions[pollutant][x]['aic'])
    
    # Perform Chi-square test on the best distribution
    params = pre_distributions[pollutant][best_dist]['params']
    chi_result = chi_square_test(pre_clean, pollutant, best_dist, params)
    
    # Store results
    chi_square_results[f"pre_{pollutant}"] = {
        'best_distribution': best_dist,
        'chi2': chi_result['chi2'],
        'p_value': chi_result['p_value']
    }

# Fit distributions to each pollutant in post-SEZ data
for pollutant in pollutants:
    post_distributions[pollutant] = fit_distributions(post_clean, pollutant)
    
    # Find best distribution based on AIC
    best_dist = min(post_distributions[pollutant], 
                    key=lambda x: post_distributions[pollutant][x]['aic'])
    
    # Perform Chi-square test on the best distribution
    params = post_distributions[pollutant][best_dist]['params']
    chi_result = chi_square_test(post_clean, pollutant, best_dist, params)
    
    # Store results
    chi_square_results[f"post_{pollutant}"] = {
        'best_distribution': best_dist,
        'chi2': chi_result['chi2'],
        'p_value': chi_result['p_value']
    }

# Display results
for key, value in chi_square_results.items():
    print(f"{key}: Best fit = {value['best_distribution']}, Chi² = {value['chi2']:.2f}, p = {value['p_value']:.4f}")

pre_PM2.5 (µg/m³): Best fit = Gamma, Chi² = 29.24, p = 0.0001
pre_PM10 (µg/m³): Best fit = Weibull, Chi² = 18.58, p = 0.0049
pre_NO (µg/m³): Best fit = Lognormal, Chi² = 34.45, p = 0.0000
pre_NO2 (µg/m³): Best fit = Gamma, Chi² = 82.41, p = 0.0000
pre_CO (mg/m³): Best fit = Lognormal, Chi² = 21.65, p = 0.0014
pre_SO2 (µg/m³): Best fit = Lognormal, Chi² = 98.75, p = 0.0000
pre_Ozone (µg/m³): Best fit = Lognormal, Chi² = 8.17, p = 0.2262
post_PM2.5 (µg/m³): Best fit = Lognormal, Chi² = 13.70, p = 0.0332
post_PM10 (µg/m³): Best fit = Gamma, Chi² = 4.82, p = 0.5675
post_NO (µg/m³): Best fit = Lognormal, Chi² = 17.88, p = 0.0065
post_NO2 (µg/m³): Best fit = Lognormal, Chi² = 19.96, p = 0.0028
post_CO (mg/m³): Best fit = Lognormal, Chi² = 28.09, p = 0.0001
post_SO2 (µg/m³): Best fit = Lognormal, Chi² = 2014.42, p = 0.0000
post_Ozone (µg/m³): Best fit = Lognormal, Chi² = 140.86, p = 0.0000


In [192]:
# Define Indian NAAQS standards (in μg/m³)
NAAQS = {

    'PM2.5 (µg/m³)': 60,  # 24-hour average

    'NO2 (µg/m³)': 80,    # 24-hour average

    'SO2 (µg/m³)': 80,     # 24-hour average

    'PM10 (µg/m³)': 100,  # 24-hour average

    'NO (µg/m³)': 80,    # 8-hour average

    'CO (mg/m³)': 4.0,     # 8-hour average

    'Ozone (µg/m³)': 100      # 8-hour average (Ozone)

} 


# Calculate exceedance rates for pre-SEZ
pre_exceedance = {}
for pollutant in pollutants:
    exceedance_count = (pre_clean[pollutant] > NAAQS[pollutant]).sum()
    exceedance_percent = (exceedance_count / len(pre_clean)) * 100
    pre_exceedance[pollutant] = {
        'count': exceedance_count,
        'percent': exceedance_percent
    }

# Calculate exceedance rates for post-SEZ
post_exceedance = {}
for pollutant in pollutants:
    exceedance_count = (post_clean[pollutant] > NAAQS[pollutant]).sum()
    exceedance_percent = (exceedance_count / len(post_clean)) * 100
    post_exceedance[pollutant] = {
        'count': exceedance_count,
        'percent': exceedance_percent
    }

# Print exceedance results
print("\nNAAQS Exceedance Analysis:")
print("-" * 50)
print(f"{'Pollutant':<10} | {'Pre-SEZ (2005)':<20} | {'Post-SEZ (2007)':<20}")
print("-" * 50)
for pollutant in pollutants:
    pre_pct = pre_exceedance[pollutant]['percent']
    post_pct = post_exceedance[pollutant]['percent']
    print(f"{pollutant.split()[0]:<10} | {pre_pct:.1f}% ({pre_exceedance[pollutant]['count']} days) | "
          f"{post_pct:.1f}% ({post_exceedance[pollutant]['count']} days)")


NAAQS Exceedance Analysis:
--------------------------------------------------
Pollutant  | Pre-SEZ (2005)       | Post-SEZ (2007)     
--------------------------------------------------
PM2.5      | 69.3% (253 days) | 85.8% (313 days)
PM10       | 82.7% (302 days) | 84.7% (309 days)
NO         | 0.3% (1 days) | 2.2% (8 days)
NO2        | 0.0% (0 days) | 1.9% (7 days)
CO         | 0.0% (0 days) | 1.9% (7 days)
SO2        | 0.0% (0 days) | 0.0% (0 days)
Ozone      | 0.0% (0 days) | 0.0% (0 days)


In [193]:
def plot_distribution_fit(data, pollutant, period, distributions_dict, NAAQS):
    """
    Plot histogram of observed data with fitted distributions.
    Now handles missing data, auto-scales, and is safe against key errors.
    """
    # Check if pollutant exists in distribution results
    if pollutant not in distributions_dict:
        print(f"⚠️ Pollutant '{pollutant}' not found in provided distribution results for {period}. Skipping.")
        return
    
    values = data[pollutant].dropna().values  # Remove NaN
    
    # Check if any valid data
    if len(values) == 0:
        print(f"⚠️ No data available for '{pollutant}' ({period}). Skipping.")
        return
    
    # Get distribution fitting results
    dist_results = distributions_dict[pollutant]
    
    # Find best distribution by minimum AIC
    best_dist = min(dist_results, key=lambda x: dist_results[x]['aic'])
    
    # Smart x range
    try:
        lower, upper = np.percentile(values, [1, 99])
        if lower == upper:  # fallback in case percentile collapsed
            lower = np.min(values)
            upper = np.max(values)
    except Exception as e:
        print(f"⚠️ Problem calculating percentiles for {pollutant}: {e}")
        return
    
    x = np.linspace(lower, upper, 1000)
    
    # Create plot
    plt.figure(figsize=(12, 6))
    
    # Plot histogram of observed data
    plt.hist(values, bins='auto', density=True, alpha=0.6, color='orange', label='Observed Data')
    
    # Plot best-fit distribution
    params = dist_results[best_dist]['params']
    
    if best_dist == 'Lognormal':
        y = stats.lognorm.pdf(x, *params)
    elif best_dist == 'Weibull':
        y = stats.weibull_min.pdf(x, *params)
    elif best_dist == 'Gamma':
        y = stats.gamma.pdf(x, *params)
    else:
        print(f"⚠️ Unknown best distribution '{best_dist}' for {pollutant}. Skipping.")
        return
    
    plt.plot(x, y, 'b-', linewidth=2, label=f'{best_dist} (Best Fit)')
    
    # Plot other candidate distributions (thinner lines)
    line_styles = ['--', ':']  # dashed and dotted
    style_index = 0
    
    for dist_name, dist_info in dist_results.items():
        if dist_name == best_dist:
            continue
        params = dist_info['params']
        if dist_name == 'Lognormal':
            y = stats.lognorm.pdf(x, *params)
        elif dist_name == 'Weibull':
            y = stats.weibull_min.pdf(x, *params)
        elif dist_name == 'Gamma':
            y = stats.gamma.pdf(x, *params)
        else:
            continue  # skip unknown
    
        line_style = line_styles[style_index % len(line_styles)]
        plt.plot(x, y, line_style, linewidth=1.5, alpha=0.8, label=dist_name)
        style_index += 1

    
    # Plot NAAQS limit if available
    if pollutant in NAAQS:
        plt.axvline(x=NAAQS[pollutant], color='black', linestyle='-.', label=f'NAAQS ({NAAQS[pollutant]} μg/m³)')
    
    # Labels and title
    plt.xlabel(pollutant)
    plt.ylabel('Probability Density')
    plt.title(f'{period}: {pollutant} Distribution Fit')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.xlim(lower, upper)  # smart limits
    plt.tight_layout()
    
    # Save plot safely
    safe_name = pollutant.replace(" ", "_").replace("/", "_")
    plt.savefig(f'{period.lower().replace("-", "_")}_{safe_name}_distribution.png', dpi=300)
    plt.close()
    
    print(f"✅ Finished plot for {pollutant} ({period}).")




# For pre-SEZ period
for pollutant in pollutants:
    plot_distribution_fit(pre_clean, pollutant, 'Pre-SEZ', pre_distributions, NAAQS)

# For post-SEZ period
for pollutant in pollutants:
    plot_distribution_fit(post_clean, pollutant, 'Post-SEZ', post_distributions, NAAQS)


✅ Finished plot for PM2.5 (µg/m³) (Pre-SEZ).
✅ Finished plot for PM10 (µg/m³) (Pre-SEZ).
✅ Finished plot for NO (µg/m³) (Pre-SEZ).
✅ Finished plot for NO2 (µg/m³) (Pre-SEZ).
✅ Finished plot for CO (mg/m³) (Pre-SEZ).
✅ Finished plot for SO2 (µg/m³) (Pre-SEZ).
✅ Finished plot for Ozone (µg/m³) (Pre-SEZ).
✅ Finished plot for PM2.5 (µg/m³) (Post-SEZ).
✅ Finished plot for PM10 (µg/m³) (Post-SEZ).
✅ Finished plot for NO (µg/m³) (Post-SEZ).
✅ Finished plot for NO2 (µg/m³) (Post-SEZ).
✅ Finished plot for CO (mg/m³) (Post-SEZ).
✅ Finished plot for SO2 (µg/m³) (Post-SEZ).
✅ Finished plot for Ozone (µg/m³) (Post-SEZ).


In [194]:
# Calculate percentage change in pollutant concentrations and include additional statistics
percent_change = {}
median_change = {}
percentile25_change = {}

for pollutant in pollutants:
    pre_mean = pre_clean[pollutant].mean()
    post_mean = post_clean[pollutant].mean()
    pre_median = pre_clean[pollutant].median()
    post_median = post_clean[pollutant].median()
    

    percent_change[pollutant] = ((post_mean - pre_mean) / pre_mean) * 100
    median_change[pollutant] = ((post_median - pre_median) / pre_median) * 100


# Create a comprehensive results dataframe
results_data = []
for pollutant in pollutants:
    pre_best = chi_square_results[f"pre_{pollutant}"]['best_distribution']
    post_best = chi_square_results[f"post_{pollutant}"]['best_distribution']
    
    results_data.append({
        'Pollutant': pollutant.split()[0],
        'Pre-SEZ Mean': pre_clean[pollutant].mean(),
        'Post-SEZ Mean': post_clean[pollutant].mean(),
        'Change (%) (Mean)': percent_change[pollutant],
        'Pre-SEZ Median': pre_clean[pollutant].median(),
        'Post-SEZ Median': post_clean[pollutant].median(),
        'Change (%) (Median)': median_change[pollutant],
        'Pre-SEZ Distribution': pre_best,
        'Post-SEZ Distribution': post_best,
        'Distribution Changed': pre_best != post_best,
        'Pre-SEZ Exceedance (%)': pre_exceedance[pollutant]['percent'],
        'Post-SEZ Exceedance (%)': post_exceedance[pollutant]['percent'],
        'Exceedance Change (pp)': post_exceedance[pollutant]['percent'] - pre_exceedance[pollutant]['percent']
    })

# Convert to DataFrame
results_df = pd.DataFrame(results_data)

# Save results
results_df.to_csv('noida_sez_impact_results.csv', index=False)

# Display
print("\nFinal Results Summary:")
print("-" * 120)
print(results_df.to_string(index=False))
print("-" * 120)

# Bar chart for mean change
plt.figure(figsize=(12, 6))
colors = ['green' if x < 0 else 'red' for x in results_df['Change (%) (Mean)'].values]
plt.bar(results_df['Pollutant'], results_df['Change (%) (Mean)'], color=colors)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.title('Percentage Change in Pollutant Mean Concentrations (Post-SEZ vs Pre-SEZ)')
plt.ylabel('Change (%) (Mean)')
plt.grid(axis='y', alpha=0.3)
for i, v in enumerate(results_df['Change (%) (Mean)']):
    plt.text(i, v + (5 if v > 0 else -5), f"{v:.1f}%", ha='center', va='center')
plt.savefig('pollutant_concentration_changes_mean.png', dpi=300)
plt.close()



Final Results Summary:
------------------------------------------------------------------------------------------------------------------------
Pollutant  Pre-SEZ Mean  Post-SEZ Mean  Change (%) (Mean)  Pre-SEZ Median  Post-SEZ Median  Change (%) (Median) Pre-SEZ Distribution Post-SEZ Distribution  Distribution Changed  Pre-SEZ Exceedance (%)  Post-SEZ Exceedance (%)  Exceedance Change (pp)
    PM2.5    111.169425     115.695926           4.071715           86.75       108.760417            25.372238                Gamma             Lognormal                  True               69.315068                85.753425               16.438356
     PM10    218.481671     196.406607         -10.103852          205.49       184.359420           -10.283021              Weibull                 Gamma                  True               82.739726                84.657534                1.917808
       NO      8.947671      19.412083         116.951237            7.09        10.514762            48.